<a href="https://colab.research.google.com/github/4may/Transformer/blob/master/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer

2018年にGoogle Brainから発表された新しいNNアーキテクチャ。attentionを使った構造が肝になる。

RNNと比較した時の、attentionのメリットは以下の通り。

* 並列計算可能であるため、GPUの恩恵を受けやすい
* データの時空間関係に依存せず、離れた位置にある情報同士の関係性も表現できる
* RNNのようなループ構造をもたず、構造がシンプル

逆に、デメリットは以下の通り。

* 時系列データの場合、ある時刻tの出力を計算をするのに全ての時刻の入力を必要とするため、非効率な場合がある

## Attention

Transformerを学ぶ前に、先にAttentionを学んでおこう。ここでは、attentionを使って日本語を英語に翻訳してみよう。

### 必要なモジュールのインポート

In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

### データの準備

データの前処理として、以下のことをしよう。

1. 各文にstart/end tokenを付け加える。
2. 各文から特殊文字を取り除く。
3. 単語と、単語のindexの変換辞書を作る
4. 各文をpaddingする

### 形態素解析

翻訳辞書内の日本語は分かち書きされていないため、形態素解析が必要。

google colab上に、辞書と必要なモジュールをインストールする。

In [32]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [33]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :   

動作確認してみよう。print("input Language: index to word mapping")
convert(input_lang, input_tensor_train[0])

In [98]:
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
sub = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
path = (sub.communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))
test = m.parse('今日は、良い気分になれる日だ！')
print(test)

今日	名詞,副詞可能,*,*,*,*,今日,キョウ,キョー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、
良い	形容詞,自立,*,*,形容詞・アウオ段,基本形,良い,ヨイ,ヨイ
気分	名詞,一般,*,*,*,*,気分,キブン,キブン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
なれる	動詞,自立,*,*,一段,基本形,なれる,ナレル,ナレル
日	名詞,非自立,副詞可能,*,*,*,日,ヒ,ヒ
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
！	記号,一般,*,*,*,*,！,！,！
EOS



In [99]:
lines = m.parse('今日は、良い気分になれる日だ！').split('\n')
#EOF, 空文字はスキップ
tokens = [line.split('\t')[0] for line in lines[:-2]]
tokens

['今日', 'は', '、', '良い', '気分', 'に', 'なれる', '日', 'だ', '！']

In [0]:
def mecab_tokenize(text):
  lines = m.parse(text).split('\n')
  tokens = [line.split('\t')[0] for line in lines[:-2]]
  return tokens

これで、分かち書きの準備ができた。

#### 翻訳辞書をダウンロード

[辞書のDL先](http://www.manythings.org/anki/)

In [14]:
#ローカル辞書をアップロードする
#!unzip jpn-eng.zip

Archive:  jpn-eng.zip
  inflating: jpn.txt                 
  inflating: _about.txt              


In [0]:
path_to_file = 'jpn.txt'

In [0]:
def unicode_to_ascii(s):
  #日本語の場合、ユニコードカテゴリMnに該当する文字もnormalizeする必要がある。
  #そうでなければ、濁点や丸点まで消えてしまう。
  #return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  return ''.join(c for c in unicodedata.normalize('NFD', s))

def preprocess_sentence(w):
  #形態素解析
  tokens = mecab_tokenize(w)
  w = ' '.join(tokens)

  #小文字に変え、かつ文の先頭と末尾の空白と改行を削除する。
  w = unicode_to_ascii(w.lower().strip())

  #単語と句読点との間にスペースを作る
  #彼 は、少年 です。 -> 彼 は 、 少年 です 。
  w = re.sub(r"([?.!,、。])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = w.strip()

  #<start> token、<end> tokenを付け加える
  #モデルの予測の開始場所と終了場所がわかるようにする。
  w = '<start> ' + w + ' <end>'
  return w

In [92]:
en_sentence = u"May I borrow this book?"
jp_sentence = u"この本を、借りてもいいですか?"
preprocess_sentence(en_sentence)

'<start> may i borrow this book ? <end>'

In [93]:
preprocess_sentence(jp_sentence)

'<start> この 本 を 、 借りて も いい です か ? <end>'

In [0]:
def create_dataset(path, num_examples):
  # 翻訳辞書の中身は以下のようになっている。
  # 最後の注釈はいらないので、落とす。
  # 英語\t日本語\t注釈\n
  # Go.	行け。	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #7421985 (Ninja)
  # Go.	行きなさい。	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #7421986 (Ninja)
  # Hi.	こんにちは。	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #373351 (tommy_san)
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:-1]] for l in lines[:num_examples]]
  
  return zip(*word_pairs)

In [95]:
en, jp = create_dataset(path_to_file, None)
en[-1]

"<start> i do many things at the same time , so not only am i reading things by akutagawa , i've also increased the amount of time i spend reading in english and i also read a little in german every day . <end>"

In [96]:
jp[-1]

'<start> 色々 並行 し て やっ てる から 芥川 ばかり 読ん でる の で も ない の だ よ 。 今 は 英語 読ん でる 時間 が 増え てる 。 ドイツ語 も 毎日 少し ずつ やっ てる 。 <end>'

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  tartget_lang, input_lang = create_dataset(path, num_examples)

  input_tensor, input_lang_tokenizer = tokenize(input_lang)
  target_tensor, tartget_lang_tokenizer = tokenize(tartget_lang)

  return input_tensor, target_tensor, input_lang_tokenizer, tartget_lang_tokenizer

## 実験の高速化のため、データセットのサイズに制限を設ける

100,000データだと多いので、30,000にしよう。

In [0]:
'''
データ数を絞る
'''
num_examples = 30000
input_tensor, target_tensor, input_lang, target_lang = load_dataset(path_to_file, num_examples)

In [104]:
'''
target tensorの最大長を取得
'''
max_length_target = target_tensor.shape[1]
max_length_input = input_tensor.shape[1]
max_length_target

16

trainingデータとvalidationデータを8:2の割合で分けてみよう

In [105]:
input_tensor_train, input_tensor_valid, target_tensor_train, target_tensor_valid = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_valid), len(target_tensor_valid))

24000 24000 6000 6000


次に、単語と単語インデックスのマッピング関数を定義しよう。Attention layerは文字列ではなく数値を入力にとるため、文字列の数値変換が必要になる。

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print("%d ------> %s" % (t, lang.index_word[t]))

In [109]:
print("Input Language: index to word mapping")
convert(input_lang, input_tensor_train[0])
print()
print("Target Language: index to word mapping")
convert(target_lang, target_tensor_train[0])

Input Language: index to word mapping
1 ------> <start>
3502 ------> リンカーン
4 ------> は
9531 ------> 1865年
7 ------> に
9532 ------> 暗殺
91 ------> さ
75 ------> れ
5 ------> た
3 ------> 。
2 ------> <end>

Target Language: index to word mapping
1 ------> <start>
2657 ------> lincoln
25 ------> was
5829 ------> assassinated
18 ------> in
3244 ------> 1865
3 ------> .
2 ------> <end>


tensorflowのDatasetオブジェクトを作ろう。

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_input_size = len(input_lang.word_index) + 1
vocab_target_size = len(target_lang.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [111]:
ex_input_batch, ex_target_batch = next(iter(dataset))
ex_input_batch.shape, ex_target_batch.shape

(TensorShape([64, 27]), TensorShape([64, 16]))

これで、Attentionに渡すデータの準備ができた。